In [1]:
import os
from dotenv import load_dotenv

from langchain_community.tools import WikipediaQueryRun, ArxivQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_ollama import OllamaLLM
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain import hub

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
load_dotenv()

True

In [20]:
wiki_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=wiki_wrapper)
print(wiki.invoke("AI"))

Page: Artificial intelligence
Summary: Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasonin


In [4]:
loader = WebBaseLoader("https://docs.langchain.com/langsmith/home")
docs = loader.load()
doc_splits = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap = 200).split_documents(docs)
vector_db = FAISS.from_documents(doc_splits, OpenAIEmbeddings())
retriever = vector_db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000020CBFB699D0>, search_kwargs={})

In [5]:
retrieval_tool = create_retriever_tool(retriever, "langsmith_search",
                                       "Search for information about LangSmith, for any langsmith search you must use this tool")

In [6]:
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [7]:
tools = [wiki, arxiv, retrieval_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\shaur\\Desktop\\Learnings\\Langchain\\lang_env\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about LangSmith, for any langsmith search you must use this tool', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000020CBDF4B7E0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langcha

In [11]:
llm = ChatOpenAI(model="gpt-5-nano", temperature=0)
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [12]:
### Agents
agent = create_openai_tools_agent(llm=llm, tools=tools, prompt=prompt)

## Agent Executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [13]:
agent_executor.invoke({
    "input": "Tell me about langsmith"
})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'LangSmith LangChain'}`


LangSmith is framework agnostic — you can use it with or without LangChain’s open source frameworks langchain and langgraph.

Get started with LangSmith - Docs by LangChainOur new LangChain Academy course on Deep Agents is now live! Enroll for free.Docs by LangChain home pagePythonSearch...⌘KLangSmithPlatform for LLM observability and evaluationOverviewQuickstartsTrace an applicationEvaluate an applicationTest promptsAPI & SDKsAPI referencePython SDKJS/TS SDKPricingPlansPricing FAQOur new LangChain Academy course on Deep Agents is now live! Enroll for free.Docs by LangChain home pagePythonSearch...⌘KGitHubForumForumSearch...NavigationGet started with LangSmithGet startedObservabilityEvaluationPrompt engineeringSelf-hostingAdministrationGet startedObservabilityEvaluationPrompt engineeringSelf-hostingAdministrationGitHubForumGet started with LangSmithCopy pageCopy pageLangSmith

{'input': 'Tell me about langsmith',
 'output': 'LangSmith is a platform for building and operating production-grade LLM applications. It focuses on observability, evaluation, and collaboration to help you ship faster and more reliably.\n\nKey ideas\n- Framework-agnostic: can be used with LangChain and its ecosystem, but also works with non-LangChain setups. It’s designed to be platform-agnostic.\n- Observability: trace and gain visibility into each step of a request to debug faster.\n- Evaluation: measure quality over time, track metrics, and compare changes as you iterate.\n- Prompt testing and versioning: test prompts, manage prompt versions, and collaborate with your team.\n- Collaboration and governance: set up a workspace, configure admin settings, invite teammates, and implement access controls.\n- Self-hosting option: you can self-host LangSmith if you prefer or need to run it on your own infrastructure.\n- SDKs and APIs: available Python and JavaScript/TypeScript SDKs (with a 

In [14]:
agent_executor.invoke({
    "input": "Tell me about machine learning"
})



> Entering new AgentExecutor chain...
Here’s a concise primer on machine learning (ML).

What it is
- ML is a set of methods that enable computers to learn patterns from data and make predictions or decisions without being explicitly programmed for each task.
- It’s about learning rules from examples rather than writing explicit code.

Key ideas
- Instead of hard-coding answers, you train a model on data so it can generalize to new, unseen data.
- The quality of ML depends on data quality, the choice of model, and how well you evaluate and tune it.

Main types of ML
- Supervised learning: learn from labeled examples. Tasks include:
  - Classification (e.g., spam detection, image type labeling)
  - Regression (e.g., predicting house prices, weather temperatures)
- Unsupervised learning: find structure in unlabeled data. Tasks include:
  - Clustering (e.g., customer segmentation)
  - Dimensionality reduction (e.g., reducing features for visualization)
- Semi-supervised learning: uses a

{'input': 'Tell me about machine learning',
 'output': 'Here’s a concise primer on machine learning (ML).\n\nWhat it is\n- ML is a set of methods that enable computers to learn patterns from data and make predictions or decisions without being explicitly programmed for each task.\n- It’s about learning rules from examples rather than writing explicit code.\n\nKey ideas\n- Instead of hard-coding answers, you train a model on data so it can generalize to new, unseen data.\n- The quality of ML depends on data quality, the choice of model, and how well you evaluate and tune it.\n\nMain types of ML\n- Supervised learning: learn from labeled examples. Tasks include:\n  - Classification (e.g., spam detection, image type labeling)\n  - Regression (e.g., predicting house prices, weather temperatures)\n- Unsupervised learning: find structure in unlabeled data. Tasks include:\n  - Clustering (e.g., customer segmentation)\n  - Dimensionality reduction (e.g., reducing features for visualization)\n-